In [ ]:
import helpers as hlp
import plotting as aplt
import iris
import iris.quickplot as qplt
import iris.plot as iplt
import matplotlib.pyplot as plt
import control_experiment_1 as experiment_runner
from pathlib import Path

In [ ]:
plotting_output_directory = Path("plots/control_experiment_1")
plotting_output_directory.mkdir(exist_ok=True)

In [ ]:
naive_exp_ids = ["C1N0", "C1N1", "C1N2"]
schwarz_exp_ids = ["C1S0", "C1S1", "C1S2"]
max_schwarz_iters = experiment_runner.max_iters

In [ ]:
def load_variables(setup: str, exp_ids: list):
    oce_t_files = [f"{setup}/{exp_id}/{exp_id[:4]}*_T.nc" for exp_id in exp_ids]
    atm_prog_files = [f"{setup}/{exp_id}/progvar.nc" for exp_id in exp_ids]
    atm_diag_files = [f"{setup}/{exp_id}/diagvar.nc" for exp_id in exp_ids]
    atm_temps = [
        hlp.load_cube(atm_prog_file, "Temperature") for atm_prog_file in atm_prog_files
    ]
    oce_ssts = [
        hlp.load_cube(oce_t_file, "Sea Surface temperature")
        for oce_t_file in oce_t_files
    ]
    atm_ssws = [
        hlp.load_cube(atm_diag_file, "Surface SW Radiation")
        for atm_diag_file in atm_diag_files
    ]
    return atm_temps, oce_ssts, atm_ssws

In [ ]:
setup = "PAPA"
atm_temps, oce_ssts, atm_ssws = load_variables(setup, naive_exp_ids)

colors = ["m", "c", "y"]
labels = ["parallel", "atm-first", "oce-first"]
alpha = 0.7
linestyles = ["-", "-", "-"]

fig, axs = plt.subplots(3, 1)
fig.set_size_inches(15, 10)
fig.suptitle(f"{naive_exp_ids[0]}, {naive_exp_ids[1]}, {naive_exp_ids[2]}", y=0.95, size=14)

aplt.create_atm_temps_plot(axs[0], atm_temps, colors, alpha, labels, linestyles)
aplt.create_oce_ssts_plot(axs[1], oce_ssts, colors, alpha, labels, linestyles)
aplt.create_atm_ssws_plot(axs[2], atm_ssws, colors, alpha, labels, linestyles)
fig.savefig(
    plotting_output_directory / "naive_coupling_schemes.pdf",
    bbox_inches="tight",
)

In [ ]:
setup = "PAPA"
final_iters = [f"{exp_id}_{max_schwarz_iters}" for exp_id in schwarz_exp_ids]
atm_temps, oce_ssts, atm_ssws = load_variables(setup, final_iters)

colors = ["m", "c", "y"]
labels = ["parallel", "atm-first", "oce-first"]
alpha = 0.7
linestyles = ["-", "-", "-"]

fig, axs = plt.subplots(3, 1)
fig.set_size_inches(15, 10)
fig.suptitle(f"Final Schwarz Iterations for Control Experiment 1", y=0.95, size=14)

aplt.create_atm_temps_plot(axs[0], atm_temps, colors, alpha, labels, linestyles)
aplt.create_oce_ssts_plot(axs[1], oce_ssts, colors, alpha, labels, linestyles)
aplt.create_atm_ssws_plot(axs[2], atm_ssws, colors, alpha, labels, linestyles)
fig.savefig(
    plotting_output_directory / "schwarz_coupling.pdf",
    bbox_inches="tight",
)

# Naive vs. Schwarz Solution

In [ ]:
setup = "PAPA"
atm_temps, oce_ssts, atm_ssws = load_variables(setup, [*naive_exp_ids, f"{schwarz_exp_ids[0]}_{max_schwarz_iters}"])

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atmosphere-first", "ocean-first", "converged SWR"]
alpha = 1
linestyles = ["--", ":", "-.", "-"]

fig, axs = plt.subplots(3, 1)
fig.set_size_inches(15, 10)
fig.suptitle(f"Non-iterative vs. Schwarz (CE1)", y=0.95, size=14)

aplt.create_atm_temps_plot(axs[0], atm_temps, colors, alpha, labels, linestyles)
aplt.create_oce_ssts_plot(axs[1], oce_ssts, colors, alpha, labels, linestyles)
aplt.create_atm_ssws_plot(axs[2], atm_ssws, colors, alpha, labels, linestyles)
axs[0].legend(framealpha=1, fancybox=False)
fig.savefig(
    plotting_output_directory / "coupling_comparison.pdf",
    bbox_inches="tight",
)

# Boundary layer type

In [ ]:
atm_diag_files = [f"PAPA/{exp_id}/diagvar.nc" for exp_id in naive_exp_ids]
atm_diag_files.append(f"PAPA/{schwarz_exp_ids[0]}_{max_schwarz_iters}/diagvar.nc")
varname = "pbl_type"
pbl_types = [hlp.load_cube(file, varname) for file in atm_diag_files]

swr_pbl_type = pbl_types[-1]
for pbl_type in pbl_types[:-1]:
    iris.util.mask_cube(pbl_type, (pbl_type.data - swr_pbl_type.data == 0))

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atm-first", "oce-first", "converged SWR"]
alpha = 0.7
markers = [".", "1", "+", "."]

fig, ax = plt.subplots()
fig.set_size_inches(10, 6)
fig.suptitle(f"Boundary Layer Type in Control Experiment 1", y=0.95, size=14)

for i in range(len(colors)):
    pbl_type = pbl_types[i]
    color = colors[i]
    label = labels[i]
    marker = markers[i]
    time_coord = pbl_type.coord("time")
    new_time_coord = iris.coords.DimCoord(
        time_coord.points,
        standard_name="time",
        long_name="Time",
        var_name="time",
        units="seconds since 2014-07-01 00:00:00",
    )
    # time shift: -7h from UTC to PDT
    new_time_coord.points = new_time_coord.points - 7 * 3600
    pbl_type.remove_coord("time")
    pbl_type.add_dim_coord(new_time_coord, 0)
    iplt.plot(
        pbl_type,
        axes=ax,
        color=color,
        label=label,
        alpha=alpha,
        ls="none",
        marker=marker,
    )
ax.set_ylabel("Boundary layer type")
ax.set_xlabel("")
ax.set_yticks([0, 1, 2, 3])
ax.set_title(r"only shows points where BL type $\neq$ SWR result")
ax.legend()
fig.savefig(
    plotting_output_directory / "pbl_type_comparison.pdf",
    bbox_inches="tight",
)

In [ ]:
atm_diag_files = [f"PAPA/{exp_id}/diagvar.nc" for exp_id in naive_exp_ids]
atm_diag_files.append(f"PAPA/{schwarz_exp_ids[0]}_{max_schwarz_iters}/diagvar.nc")
varname = "Surface Sensible Heat Flux"
sh_fluxes = [hlp.load_cube(file, varname) for file in atm_diag_files]

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atm-first", "oce-first", "converged SWR"]
alpha = 0.7
linestyles = ["--", ":", "-.", "-"]

fig, ax = plt.subplots()
fig.set_size_inches(10, 6)
fig.suptitle(f"Surface SH Flux in Control Experiment 1", y=0.95, size=14)

for i in range(len(colors)):
    sh_flux = sh_fluxes[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    time_coord = sh_flux.coord("time")
    new_time_coord = iris.coords.DimCoord(
        time_coord.points,
        standard_name="time",
        long_name="Time",
        var_name="time",
        units="seconds since 2014-07-01 00:00:00",
    )
    # time shift: -7h from UTC to PDT
    new_time_coord.points = new_time_coord.points - 7 * 3600
    sh_flux.remove_coord("time")
    sh_flux.add_dim_coord(new_time_coord, 0)
    iplt.plot(
        sh_flux,
        axes=ax,
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.set_ylabel(r"Surface SH Flux $[W m^{-2}]$")
ax.set_xlabel("")
ax.set_title("")
ax.legend()
ax.grid()
fig.savefig(
    plotting_output_directory / "ssh_comparison.pdf",
    bbox_inches="tight",
)